In [198]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import itertools
from pathlib import Path
import json
import collections

In [172]:
def parse_lyric(link):
    r = requests.get(link, proxies=proxy)
    soup = BeautifulSoup(r.text, 'lxml')
    lyric_area = soup.find('dd', id='fsZx3').strings
    lyric = []
    for idx, item in enumerate(lyric_area):
        if not (item.startswith('[00') or item.startswith('[ti:')):
            lyric.append(item)
        else:
            break
    lyric = [item.replace('\u3000', ' ').strip() for item in lyric]
    return lyric

In [177]:
df = pd.read_csv('all_jay_songs.csv')
albums = df.TAG_album.unique()

In [178]:
albums = np.append(albums, ['周杰伦的床边故事', '杰伦(Jay)', '11月的萧邦', '我很忙(ON THE RUN)', '寻找周杰伦', '惊叹号', '范特西(Fantasy)'])

In [179]:
albums

array(['Jay', '七里香', '不能说的秘密', '依然范特西', '八度空间', '十一月的萧邦', '十二新作', '叶惠美',
       '哎呦，不错哦', '寻找周杰伦EP', '惊叹号！', '我很忙', '范特西', '跨时代', '霍元甲EP', '魔杰座',
       '黄金甲EP', '周杰伦的床边故事', '杰伦(Jay)', '11月的萧邦', '我很忙(ON THE RUN)',
       '寻找周杰伦', '惊叹号', '范特西(Fantasy)'], dtype=object)

In [2]:
url = 'http://mojim.com/cnh100951.htm'

In [3]:
proxy = {'http': 'http://127.0.0.1:1080', 'https': 'http://127.0.0.1:1080'}

In [4]:
r = requests.get(url, proxies=proxy)

In [5]:
soup = BeautifulSoup(r.text, 'lxml')

In [204]:
albums_info = collections.defaultdict(dict)
for dd in soup.find_all('dd', class_=['hb3', 'hb2']):
    album = dd.find('span', class_='hc1').text.strip()
    if album in albums:
        hc2 = dd.find('span', class_='hc2')
        release_date = ''.join(re.findall('\d+', hc2.text.strip()))
        try:
            distributor = hc2.find('font').text
        except Exception as e:
            distributor = None
        albums_info[album]['date'] = release_date
        albums_info[album]['distributor'] = distributor
        
        hc34 = dd.find_all('span', class_=['hc3', 'hc4'])
        for span in hc34:
            for a in span.find_all('a', href=True, title=True):
                link = 'http://mojim.com' + a['href']
                song_name = a.text
                print(f'Processing {song_name} in {album} ...')
                try:
                    lyric = parse_lyric(link)
                except Exception as e:
                    lyric = ''
                    print(e)
                path = Path('Lyrics').joinpath(album)
                path.mkdir(parents=True, exist_ok=True)
                path.joinpath(song_name + '.txt').write_text('\n'.join(lyric), encoding='utf8')
with open('albums_info.json', 'w', encoding='utf8') as f:
    json.dump(albums_info, f, ensure_ascii=False)
#         print(album, release_date, language, distributor)

In [176]:
parse_lyric('http://mojim.com/cny100951x20x1.htm')

['作词：方文山',
 '作曲：周杰伦',
 '旌旗如虹 山堆叠如峰',
 '这军队蜿蜒如龙 杀气如风 血色如酒红',
 '将军我傲气如冲 神色悍如凶',
 '黄金甲如忠 铁骑剽悍我行如轰',
 '景色如冬 萧瑟如风 攻势如弓 魂断犹如梦中',
 '一静一动 如松 千年不变 如空 如空 如空',
 '血染盔甲 我挥泪杀',
 '满城菊花 谁的天下',
 '宫廷之上 狼烟风沙',
 '生死不过 一刀 的疤',
 '仇恨绵延如火 愁入眉头如锁',
 '情感漂泊漂泊 漂泊一世如我',
 '今生繁华如昨 兵戎相见如破',
 '千军万马万马 万马奔腾那骨头相残如错',
 '陈年战事如酒 成败转眼如秋',
 '遍地烽火烽火',
 '烽火回忆如锈  (那烽火回忆如锈)',
 '杀戮过重如否  (那杀戮过重如否)',
 '更多更详尽歌词 在',
 '※ Mojim.com 魔镜歌词网',
 '烽火回忆如锈  (那烽火回忆如锈)',
 '皇室血脉如断流 (皇室血脉如断流)',
 '血染盔甲 我挥泪杀',
 '满城菊花 谁的天下',
 '宫廷之上 狼烟风沙',
 '生死不过 一刀 的疤',
 '血染盔甲 我挥泪杀',
 '满城菊花 谁的天下',
 '宫廷之上 狼烟风沙',
 '生死不过 一刀 的疤',
 '盔甲 我挥泪杀',
 '满城菊花 谁的天下',
 '宫廷之上 狼烟风沙',
 '生死不过 一刀的疤',
 '血染盔甲 我挥泪杀',
 '横刀立马 看谁倒下',
 '爱恨对话 历史留下',
 '谁在乱箭 之中潇洒']

In [109]:
rr_soup = BeautifulSoup(rr.text, 'lxml')